In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import sys
sys.path.append('/home/youyou/workspace/FinRL-Tutorials')
%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/home/youyou/anaconda3/envs/FinEnv/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [26]:
"""Fetches data from Yahoo API
        Parameters
        ----------

        Returns
        -------
        `pd.DataFrame`
            7 columns: A date, open, high, low, close, volume and tick symbol
            for the specified stock ticker
        """
        # Download and save the data in a pandas DataFrame:
data_df = pd.read_csv('DJI_test.csv')
data_df = data_df.reset_index()
data_df=data_df[['trade_date',"open","high","low","close","pre_close","vol","tic"]]
try:
        # convert the column names to standardized names
        data_df.columns = [
        "date",
        "open",
        "high",
        "low",
        "close",
        "adjcp",
        "volume",
        "tic",
        ]
        # use adjusted close price instead of close price
        data_df["close"] = data_df["adjcp"]
        # drop the adjusted close price column
        data_df = data_df.drop(labels="adjcp", axis=1)
except NotImplementedError:
        print("the features are not supported currently")
# create day of the week column (monday = 0)
data_df["day"] = data_df["date"].dt.dayofweek
# convert date to standard string format, easy to filter
data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
# drop missing data
data_df = data_df.dropna()
data_df = data_df.reset_index(drop=True)
print("Shape of DataFrame: ", data_df.shape)
# print("Display DataFrame: ", data_df.head())

data_df = data_df.sort_values(by=["date", "tic"]).reset_index(drop=True)

     ts_code  trade_date      open     close      high       low  pre_close  \
0        DJI    20231025  33203.53  33035.93  33267.91  32989.72   33141.38   
1        DJI    20231024  33089.64  33141.38  33272.19  32988.43   32936.41   
2        DJI    20231023  32993.02  32936.41  33234.85  32892.19   33127.28   
3        DJI    20231020  33365.27  33127.28  33425.77  33118.12   33414.17   
4        DJI    20231019  33669.52  33414.17  33852.39  33368.58   33665.08   
...      ...         ...       ...       ...       ...       ...        ...   
3995     DJI    20071213  13473.98  13517.96  13527.63  13353.92   13473.90   
3996     DJI    20071212  13434.80  13473.90  13704.52  13321.64   13432.77   
3997     DJI    20071211  13726.87  13432.77  13780.11  13413.74   13727.03   
3998     DJI    20071210  13623.55  13727.03  13754.02  13623.55   13625.58   
3999     DJI    20071207  13618.27  13625.58  13668.10  13601.36   13619.89   

      change  pct_chg  swing        vol  tic  
0   

/tmp/ipykernel_431311/499561038.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_df = data_df.drop(


KeyError: "['price_change', 'p_change', 'ma5', 'ma10', 'ma20', 'v_ma5', 'v_ma10', 'v_ma20'] not found in axis"

In [34]:
import tushare as ts
pro = ts.pro_api()
ts.set_token('27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5')
df = pro.daily('DJI', start='20220103',end='20230103')


Exception: 没有可查询字段

In [31]:
df

AttributeError: 'NoneType' object has no attribute 'columns'

In [14]:
df_1 = pro.index_global(ts_code='DJI', start_date='20220103', end_date='20230103')
df_1

,ts_code,trade_date,open,close,high,low,pre_close,change,pct_chg,swing,vol
0,DJI,20230103,33148.90,33136.37,33387.52,32850.57,33147.25,-10.88,-0.03,1.62,358608.32
1,DJI,20221230,33121.61,33147.25,33152.55,32847.82,33220.80,-73.55,-0.22,0.92,296192.00
2,DJI,20221229,33021.43,33220.80,33293.42,33020.35,32875.71,345.09,1.05,0.83,245838.13
3,DJI,20221228,33264.76,32875.71,33379.55,32869.15,33241.56,-365.85,-1.10,1.54,254303.63
4,DJI,20221227,33224.23,33241.56,33387.72,33069.58,33203.93,37.63,0.11,0.96,246014.63
...,...,...,...,...,...,...,...,...,...,...,...
247,DJI,20220107,36249.59,36231.66,36382.84,36111.53,36236.47,-4.81,-0.01,0.75,361423.38
248,DJI,20220106,36409.05,36236.47,36464.19,36200.68,36407.11,-170.64,-0.47,0.72,390187.50
249,DJI,20220105,36722.60,36407.11,36952.65,36400.39,36799.65,-392.54,-1.07,1.50,468455.38
250,DJI,20220104,36636.00,36799.65,36934.84,36636.00,36585.06,214.59,0.59,0.82,435084.32
